# patent-client

In [12]:
from patent_client import USApplication

In [15]:
aapl_apps = USApplication.objects.filter(first_named_applicant='bristol myers')

In [16]:
len(aapl_apps)

2286

In [ ]:
len(patented_app)

In [149]:
temp = aapl_apps[35]

In [155]:
temp_fields = [f.name for f in temp.fields()]

In [100]:
temp.app_filing_date

datetime.date(2015, 4, 17)

# pypatent

In [1]:
import pypatent

In [2]:
temp = pypatent.Search(an='bristol myers', 
                       #ptad='1/1/2010->12/31/2019', 
                       get_patent_details=True, 
                       results_limit=2
                      )

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
temp = temp.as_list()

In [ ]:
len(temp)

In [ ]:
for t in temp:
    print(t['applicant_name'])

In [82]:
temp[-1].keys()

dict_keys(['title', 'patent_num', 'patent_date', 'file_date', 'abstract', 'inventors', 'applicant_name', 'applicant_city', 'applicant_state', 'applicant_country', 'assignee_name', 'assignee_loc', 'family_id', 'applicant_num', 'claims', 'description', 'url'])

In [13]:
import pandas as pd
import wrds
import os
from mlf import TICK_NAME

In [14]:
db = wrds.Connection(wrds_username=os.environ['WRDS_USER'], 
                     wrds_password=os.environ['WRDS_PASS'])

Loading library list...
Done


In [3]:
ticker_query = "'" + '\', \''.join(list(TICK_NAME.keys())) + "'"

ticker_query

"'BMY', 'VRTX', 'GILD', 'RHHBY', 'AMGN', 'JNJ', 'NVO', 'ABBV', 'PFE', 'AZN', 'BIIB', 'MRK', 'GSK', 'NVS', 'REGN', 'LLY', 'ALXN'"

In [15]:
start_sql = '2010-01-01'
end_sql = '2020-11-30'

In [5]:
# get stock data
stock_data = db.raw_sql(f"select permco, ticker from crsp.stocknames where ticker in "
                        f"({ticker_query})")
stock_permco = stock_data['permco'].unique().astype(str)
print(stock_permco)
print(len(stock_permco))

['54287.0' '216.0' '20331.0' '21394.0' '21018.0' '21188.0' '23238.0'
 '5794.0' '21102.0' '3230.0' '1973.0' '10876.0' '11042.0' '11112.0'
 '11300.0' '29908.0' '5299.0' '14394.0' '37497.0']
19


In [6]:
# get stock returns
data = db.raw_sql(f"select permco, date, ret from crsp.dsf where permco in "
                  f"({', '.join(stock_permco)}) and date >= '{start_sql}' and date <= '{end_sql}'"
                  )

In [7]:
data = data.set_index('date')

In [8]:
# declare X
X = pd.DataFrame()
for permco, df in data.groupby('permco'):
    df = df.drop(columns='permco')
    ticker = stock_data[stock_data['permco']==permco]['ticker'].values[0]
    df.columns = [ticker]
    X = pd.concat([X, df], axis=1, ignore_index=False)

In [9]:
X

,AMGN,GSK,NVO,REGN,VRTX,BIIB,GILD,ALXN,BMY,JNJ,LLY,MRK,PFE,AZN,NVS,ABBV
2010-01-04,0.020246,0.017041,0.028034,0.018610,0.032439,0.002617,0.000693,-0.011676,0.015049,0.004192,0.003080,0.012863,0.040682,0.010013,-0.033437,NaN
2010-01-05,-0.008582,-0.026530,-0.014625,0.019082,-0.033002,-0.004847,-0.000924,-0.014300,-0.015607,-0.011596,-0.017588,0.004053,-0.014263,-0.020249,-0.000950,NaN
2010-01-06,-0.007515,-0.011714,0.011750,-0.015139,-0.017532,0.000937,0.034674,0.002523,-0.000396,0.008134,0.007389,0.013455,-0.003215,-0.007966,-0.003615,NaN
2010-01-07,-0.009157,0.003387,-0.008558,-0.006068,-0.012610,-0.008235,0.002458,0.003775,0.000793,-0.007137,-0.005078,0.001593,-0.003763,0.010851,-0.008784,NaN
2010-01-08,0.008886,-0.009161,0.011868,-0.008140,-0.020000,0.020570,-0.007355,0.001880,-0.016640,0.003438,-0.007655,-0.000530,0.008095,0.004294,0.008091,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,-0.002880,0.001067,0.005376,0.006173,0.002637,0.008323,-0.000449,-0.001630,-0.000791,-0.003483,-0.000305,-0.003270,-0.000763,-0.000401,-0.000527,-0.004432
2019-12-26,-0.001774,0.000213,0.002070,-0.004382,-0.001406,-0.007498,-0.004946,-0.012242,0.012666,-0.000685,0.000229,-0.001094,0.002291,-0.000401,-0.000105,-0.000223
2019-12-27,-0.001530,0.008101,0.002926,-0.004108,0.003269,-0.005633,-0.006628,-0.003488,0.002814,-0.000549,0.002898,0.001752,-0.001524,0.011836,0.005907,-0.007013
2019-12-30,-0.005217,-0.004441,-0.009097,0.001018,-0.009051,-0.013596,-0.015466,-0.011147,-0.006392,-0.003087,-0.001673,-0.005137,-0.010427,-0.008525,-0.008598,-0.007623


In [10]:
X.shape

(2516, 16)

In [11]:
X.columns

Index(['AMGN', 'GSK', 'NVO', 'REGN', 'VRTX', 'BIIB', 'GILD', 'ALXN', 'BMY',
       'JNJ', 'LLY', 'MRK', 'PFE', 'AZN', 'NVS', 'ABBV'],
      dtype='object')

In [16]:
start_sql = '2010-01-01'
end_sql = '2020-11-30'

In [19]:
# get snp500 returns
returns = db.raw_sql("select caldt, sprtrn from crsp.dsp500 where "
                     f"caldt >= '{start_sql}' and caldt <= '{end_sql}'"
                    )

In [20]:
returns

,caldt,sprtrn
0,2010-01-04,0.016043
1,2010-01-05,0.003116
2,2010-01-06,0.000546
3,2010-01-07,0.004001
4,2010-01-08,0.002882
...,...,...
2511,2019-12-24,-0.000195
2512,2019-12-26,0.005128
2513,2019-12-27,0.000034
2514,2019-12-30,-0.005781


In [23]:
F = pd.DataFrame(snp500_prices)
F.columns = ['date', '^GSPC']
F = snp500_prices.set_index('date')

In [24]:
F

,^GSPC
date,
2010-01-04,0.016043
2010-01-05,0.003116
2010-01-06,0.000546
2010-01-07,0.004001
2010-01-08,0.002882
...,...
2019-12-24,-0.000195
2019-12-26,0.005128
2019-12-27,0.000034
